In [188]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import time
from sklearn.feature_selection import SelectKBest

In [97]:
li = []
df1 = pd.read_csv('bitcoinsubreddit01.csv')
li.append(df1)
df2 = pd.read_csv('bitcoinsubreddit02.csv')
li.append(df2)
df3 = pd.read_csv('bitcoinsubreddit03.csv')
li.append(df3)
df4 = pd.read_csv('bitcoinsubreddit04.csv')
li.append(df4)
df5 = pd.read_csv('bitcoinsubreddit05.csv')
li.append(df5)
df6 = pd.read_csv('bitcoinsubreddit06.csv')
li.append(df6)

df = pd.concat(li, axis=0, ignore_index=True)

In [98]:
df["created_utc"] = pd.to_datetime(df["created_utc"],unit='s')

In [99]:
df["retrieved_on"] = pd.to_datetime(df["retrieved_on"],unit='s')

In [51]:
df = df[df["created_utc"] >= '02/01/2021']

In [52]:
df = df[df["created_utc"] < '07/01/2021']

In [100]:
df = df[df['body'] != "[removed]"]

In [101]:
df = df[df['body'] != "[deleted]"]

In [102]:
df["created_utc"] = pd.to_datetime(df["created_utc"].dt.date)

In [107]:
df["body"] = df["body"].astype(str)

In [108]:
df.dtypes

author_premium                bool
body                        object
controversiality             int64
created_utc         datetime64[ns]
distinguished               object
id                          object
locked                        bool
no_follow                     bool
parent_id                   object
quarantined                   bool
removal_reason             float64
retrieved_on        datetime64[ns]
score                        int64
subreddit                   object
dtype: object

In [109]:
dfbyday = df[["created_utc","id","author_premium","controversiality", "locked", "no_follow", "score", "body"]].groupby(["created_utc"]).agg({"id": ["count"], "author_premium": ["sum"], "controversiality": ["sum"], "locked" : ["sum"], "no_follow" : ["sum"], "score" : [np.mean], "body": ' '.join})

In [110]:
dfbyday = dfbyday.droplevel(axis=1, level=0).reset_index().set_axis(["Date", "CommentCount", "PremiumCount", "Controversiality", "LockedCount", "FollowCount", "Score", "Body"], axis=1, inplace=False)

In [111]:
dfbyday

,Date,CommentCount,PremiumCount,Controversiality,LockedCount,FollowCount,Score,Body
0,2021-01-01,6224,424,133,0,4048,4.661472,I understand what you're saying. Perhaps I sho...
1,2021-01-02,12380,1052,171,0,8073,4.480291,It doesn’t matter what price they are now if t...
2,2021-01-03,11732,1080,167,0,7306,4.427719,I totally agree. This is more of a push for th...
3,2021-01-04,8671,764,161,0,5948,3.543767,Yeah. The only thing I forgot to do was to use...
4,2021-01-05,7493,562,110,0,4888,4.018417,You do get relief. It's a capital loss if you ...
...,...,...,...,...,...,...,...,...
176,2021-06-26,5469,250,174,0,3805,3.158713,tldr; Bitcoin is a decentralized digital curre...
177,2021-06-27,5110,269,115,0,3523,3.556751,Lol this is funny. Fuck you Wooooooow Bro not ...
178,2021-06-28,4712,315,86,0,3273,4.041384,Crypto.com $19T market cap isn't unreasonable ...
179,2021-06-29,5374,334,164,0,3521,3.665984,Your submission has been flagged for removal b...


In [61]:
df = df[df['body'].isna() == False]

In [112]:
df['body'] = df['body'].str.lower()

In [113]:
vectorizer = CountVectorizer()

In [114]:
wordvector = vectorizer.fit_transform(dfbyday['Body'])

In [115]:
wordvector

<181x199133 sparse matrix of type '<class 'numpy.int64'>'
	with 2227227 stored elements in Compressed Sparse Row format>

In [116]:
wordvectordf = pd.DataFrame(wordvector.todense(), columns=vectorizer.get_feature_names_out())

,Date,CommentCount,PremiumCount,Controversiality,LockedCount,FollowCount,Score,Body
0,2021-01-01,6224,424,133,0,4048,4.661472,I understand what you're saying. Perhaps I sho...
1,2021-01-02,12380,1052,171,0,8073,4.480291,It doesn’t matter what price they are now if t...
2,2021-01-03,11732,1080,167,0,7306,4.427719,I totally agree. This is more of a push for th...
3,2021-01-04,8671,764,161,0,5948,3.543767,Yeah. The only thing I forgot to do was to use...
4,2021-01-05,7493,562,110,0,4888,4.018417,You do get relief. It's a capital loss if you ...
...,...,...,...,...,...,...,...,...
176,2021-06-26,5469,250,174,0,3805,3.158713,tldr; Bitcoin is a decentralized digital curre...
177,2021-06-27,5110,269,115,0,3523,3.556751,Lol this is funny. Fuck you Wooooooow Bro not ...
178,2021-06-28,4712,315,86,0,3273,4.041384,Crypto.com $19T market cap isn't unreasonable ...
179,2021-06-29,5374,334,164,0,3521,3.665984,Your submission has been flagged for removal b...


In [134]:
dfbydayvector = pd.concat([dfbyday[["Date"]], wordvectordf], axis=1)

In [135]:
dfbydayvector

,Date,00,000,0000,00000,000000,0000000,00000000,000000000,0000000000,...,𝚠𝚘𝚛𝚛𝚢,𝚢𝚎𝚝,𝚢𝚘𝚞,𝚢𝚘𝚞𝚛,𝚢𝚘𝚞𝚛𝚜𝚎𝚕𝚏,𝟭𝟮,𝟶𝚡𝚋𝙰𝟻𝚍𝟽𝟼𝟺𝟿𝚍𝙴𝚏𝟹𝟸𝟻𝚊𝙰𝟼𝚊𝚎𝟹𝙰,𝟷𝟶,𝟸𝟺,𝟿𝚟𝚡𝚓𝚣𝟿𝚗𝟺𝟸𝚚𝚢𝚚𝚜𝚜𝟹𝚗𝚚
0,2021-01-01,7,187,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-02,20,259,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-03,18,277,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,8,155,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-05,11,137,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2021-06-26,9,62,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177,2021-06-27,8,62,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,2021-06-28,2,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,2021-06-29,3,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
dfprice = pd.read_csv('BitCoinPrice.csv')

In [137]:
dfprice["Date"] = pd.to_datetime(dfprice["Date"], infer_datetime_format=True)  

In [145]:
dfprice["Change %"] = dfprice["Change %"].str.rstrip('%').astype('float') / 100.0

In [147]:
dfprice["NextDayChange"] = dfprice["Change %"].shift(-1)

In [148]:
dfprice

,Date,Price,Open,High,Low,Vol.,Change %,NextDayChange
0,2021-09-22,"43,551.6","40,660.8","43,976.7","40,585.7",93.98K,0.0713,-0.0518
1,2021-09-21,"40,651.3","43,012.3","43,619.3","39,646.8",142.55K,-0.0518,-0.0925
2,2021-09-20,"42,870.6","47,240.8","47,323.9","42,547.1",77.96K,-0.0925,-0.0221
3,2021-09-19,"47,238.7","48,299.9","48,341.9","46,866.6",41.98K,-0.0221,0.0217
4,2021-09-18,"48,306.7","47,283.9","48,761.0","47,070.5",43.64K,0.0217,-0.0097
...,...,...,...,...,...,...,...,...
260,2021-01-05,"33,991.5","32,015.4","34,414.7","30,010.5",202.13K,0.0615,-0.0284
261,2021-01-04,"32,022.6","33,016.6","33,587.5","28,204.5",255.27K,-0.0284,0.0238
262,2021-01-03,"32,958.9","32,192.9","34,755.9","32,029.6",155.21K,0.0238,0.0965
263,2021-01-02,"32,193.3","29,359.7","33,233.5","29,008.0",240.87K,0.0965,0.0142


In [163]:
dfvectorprice = pd.merge(dfbydayvector, dfprice[["Date", "NextDayChange"]], left_on="Date", right_on="Date", how="outer")

In [164]:
dfvectorprice

,Date,00,000,0000,00000,000000,0000000,00000000,000000000,0000000000,...,𝚢𝚎𝚝,𝚢𝚘𝚞,𝚢𝚘𝚞𝚛,𝚢𝚘𝚞𝚛𝚜𝚎𝚕𝚏,𝟭𝟮,𝟶𝚡𝚋𝙰𝟻𝚍𝟽𝟼𝟺𝟿𝚍𝙴𝚏𝟹𝟸𝟻𝚊𝙰𝟼𝚊𝚎𝟹𝙰,𝟷𝟶,𝟸𝟺,𝟿𝚟𝚡𝚓𝚣𝟿𝚗𝟺𝟸𝚚𝚢𝚚𝚜𝚜𝟹𝚗𝚚,NextDayChange
0,2021-01-01,7.0,187.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2021-01-02,20.0,259.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0142
2,2021-01-03,18.0,277.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0965
3,2021-01-04,8.0,155.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0238
4,2021-01-05,11.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,2021-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0160
261,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0275
262,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0080
263,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0423


In [165]:
dfvectorprice = dfvectorprice[dfvectorprice['00'].isna() == False].copy()

In [166]:
dfvectorprice

,Date,00,000,0000,00000,000000,0000000,00000000,000000000,0000000000,...,𝚢𝚎𝚝,𝚢𝚘𝚞,𝚢𝚘𝚞𝚛,𝚢𝚘𝚞𝚛𝚜𝚎𝚕𝚏,𝟭𝟮,𝟶𝚡𝚋𝙰𝟻𝚍𝟽𝟼𝟺𝟿𝚍𝙴𝚏𝟹𝟸𝟻𝚊𝙰𝟼𝚊𝚎𝟹𝙰,𝟷𝟶,𝟸𝟺,𝟿𝚟𝚡𝚓𝚣𝟿𝚗𝟺𝟸𝚚𝚢𝚚𝚜𝚜𝟹𝚗𝚚,NextDayChange
0,2021-01-01,7.0,187.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2021-01-02,20.0,259.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0142
2,2021-01-03,18.0,277.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0965
3,2021-01-04,8.0,155.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0238
4,2021-01-05,11.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2021-06-26,9.0,62.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0886
177,2021-06-27,8.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0206
178,2021-06-28,2.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0755
179,2021-06-29,3.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0058


In [167]:
conditions = [
               (dfvectorprice["NextDayChange"] <= -0.025), 
               (dfvectorprice["NextDayChange"] > -0.025) & (dfvectorprice["NextDayChange"] < 0.025),
               (dfvectorprice["NextDayChange"] >= 0.025)
             ]
values = [0, 1, 2]
dfvectorprice['ChangeGroup'] = np.select(conditions, values)

In [168]:
dfvectorprice = dfvectorprice.drop(['NextDayChange', 'Date'], axis=1)

In [169]:
dfvectorprice

,00,000,0000,00000,000000,0000000,00000000,000000000,0000000000,00000000000,...,𝚢𝚎𝚝,𝚢𝚘𝚞,𝚢𝚘𝚞𝚛,𝚢𝚘𝚞𝚛𝚜𝚎𝚕𝚏,𝟭𝟮,𝟶𝚡𝚋𝙰𝟻𝚍𝟽𝟼𝟺𝟿𝚍𝙴𝚏𝟹𝟸𝟻𝚊𝙰𝟼𝚊𝚎𝟹𝙰,𝟷𝟶,𝟸𝟺,𝟿𝚟𝚡𝚓𝚣𝟿𝚗𝟺𝟸𝚚𝚢𝚚𝚜𝚜𝟹𝚗𝚚,ChangeGroup
0,7.0,187.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,20.0,259.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,18.0,277.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,8.0,155.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,11.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,9.0,62.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
177,8.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
178,2.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
179,3.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [170]:
dfvectorprice_train, dfvectorprice_test = train_test_split(dfvectorprice, test_size=0.4)

In [171]:
dfvectorprice_test, dfvectorprice_valid = train_test_split(dfvectorprice_test, test_size=0.5)

In [173]:
X_train = dfvectorprice_train.drop(['ChangeGroup'], axis=1).to_numpy()

In [174]:
y_train = dfvectorprice_train[["ChangeGroup"]].to_numpy()

In [175]:
X_test = dfvectorprice_test.drop(['ChangeGroup'], axis=1).to_numpy()

In [176]:
y_test = dfvectorprice_test[["ChangeGroup"]].to_numpy()

In [177]:
X_valid = dfvectorprice_valid.drop(['ChangeGroup'], axis=1).to_numpy()

In [178]:
y_valid = dfvectorprice_valid[["ChangeGroup"]].to_numpy()

In [180]:
X_train.shape

(108, 199133)

In [190]:
X_train.shape

(108, 199133)

In [195]:
y_train.ravel().shape

(108,)

In [199]:
seletbest = SelectKBest(k=20).fit(X_train, y_train.ravel())

c:\users\emaibach\appdata\local\programs\python\python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [    10     11     15 ... 199130 199131 199132] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\users\emaibach\appdata\local\programs\python\python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [200]:
X_trainnew = seletbest.transform(X_train)

In [201]:
X_testnew = seletbest.transform(X_test)

In [202]:
X_validnew = seletbest.transform(X_valid)

In [203]:
root_logdir = os.path.join(os.curdir, "tensor_logs")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [205]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[20,]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(3, activation="softmax"))

In [206]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [207]:
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_trainnew, y_train, epochs=30, validation_data=(X_validnew, y_valid), callbacks=[tensorboard_cb])

Epoch 1/30
4/4 [==============================] - 2s 412ms/step - loss: 2.5035 - accuracy: 0.3889 - val_loss: 1.5781 - val_accuracy: 0.2703
Epoch 2/30
4/4 [==============================] - 0s 14ms/step - loss: 1.5266 - accuracy: 0.3148 - val_loss: 1.5369 - val_accuracy: 0.4324
Epoch 3/30
4/4 [==============================] - 0s 14ms/step - loss: 1.3641 - accuracy: 0.3519 - val_loss: 1.9889 - val_accuracy: 0.2432
Epoch 4/30
4/4 [==============================] - 0s 13ms/step - loss: 1.6709 - accuracy: 0.2778 - val_loss: 1.3192 - val_accuracy: 0.4595
Epoch 5/30
4/4 [==============================] - 0s 14ms/step - loss: 1.1207 - accuracy: 0.3889 - val_loss: 1.2861 - val_accuracy: 0.3514
Epoch 6/30
4/4 [==============================] - 0s 17ms/step - loss: 1.0490 - accuracy: 0.4259 - val_loss: 1.2242 - val_accuracy: 0.3784
Epoch 7/30
4/4 [==============================] - 0s 12ms/step - loss: 1.0471 - accuracy: 0.4537 - val_loss: 1.2225 - val_accuracy: 0.3784
Epoch 8/30
4/4 [==========

In [13]:
dfsample = df.sample(n=1000)

In [37]:
dfsample[["body", "BuySell"]].to_csv("samplecomments.csv")

0 = Buy
1 = Neutral
2 = Sell

In [26]:
dfsample["BuySell"] = 1

In [31]:
curindex = 0

In [32]:
dfsample.iloc[curindex,1]

'.01 BTC = 1 "get out of jail free card" for one major life event in 10 years....:  sudden unemployment, emergency replacement of a car, down payment on a house, attorney expenses, etc.'

In [34]:
dfsample.iloc[curindex,14]

1

In [36]:
dfsample[["body", "BuySell"]]

,body,BuySell
591440,".01 BTC = 1 ""get out of jail free card"" for on...",1
1433187,Well obviously having it set up so that you do...,1
1325169,Red Sunday they said,1
1097087,This rating is undercommented.,1
380335,"$120,000",1
...,...,...
829619,Just wanted to let him know about the bitcoin....,1
322728,There goes coinbase,1
907484,"Yep, I figured I better liquidate all my BTC. ...",1
467725,Use CB and you will only ever invest small - n...,1


In [22]:
dfsample.iloc[0, 2]

0

In [46]:
df[df.index == 499245]

,author_premium,body,controversiality,created_utc,distinguished,id,locked,no_follow,parent_id,quarantined,removal_reason,retrieved_on,score,subreddit
499245,False,65k by tomorrow EOD,0,2021-02-22 22:25:16,NaN,goe4ibj,False,True,t3_lpn1kx,False,NaN,1970-01-01,1,Bitcoin
